In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"

from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN

### EDA and Preprocessing with Pandas

In [ ]:
# Importing data from S3 bucket
url = 'https://uber-28-02-2023.s3.eu-west-3.amazonaws.com/uber-raw-data-apr14.csv'
df = pd.read_csv(url)

display(df.head())

print("Number of rows in the DataFrame: ", len(df))

- 'Date/Time' column contains the time of a client was picked up by a Uber driver
- 'Lat' and 'Lon' are the geographical coordinates of the place where the client was picked up
- 'Base': TLC base company code affiliated with the Uber pickup. In fact, unlike in other cities, a driver needs to have a licence from the TLC (Taxi and Limousin Company) to drive in New York. I will consider that this is purely administrative information and will not take this column into account for further analysis.

In [ ]:
# Checking if there are missing values in the dataframe
100*df.isnull().sum()/df.shape[0]

In [ ]:
# Checking data types in the dataframe
df.dtypes

The 'Date/Time' column contains data in object (string) format. We will convert this column to datetime data type in order to extract useful features from this columns.

In [ ]:
df["Date/Time"] = pd.to_datetime(df["Date/Time"], format='%m/%d/%Y %H:%M:%S')

print(df.dtypes)

df.head()

It looks like the time of a pickup is precise up to a minute. 
Let us check whether there are only one or more Uber pickups for a given timestamp.

In [ ]:
print("Top 10 timestamps with maximum pickups: ")
display(df['Date/Time'].value_counts().head(10))

print()
print("Ten timestamps with the minimum pickups: ")
display(df['Date/Time'].value_counts().tail(10))

At a first sight, it looks like the maximum number of pickups take place in the evening from about 17h00 to 20h30, and the least pickups take place late at night from midnight to around 5h in the morning.

Intuitively, it seems that we might be able to see patterns in pickup points depending on the day of the week and on the hour of the day. Let us use time series to see if the number of pickups is cyclical in nature. If it is, it might be useful for further analysis of the data.

In [ ]:
pickup_time_df = df.groupby(by = "Date/Time").size().to_frame().reset_index()
pickup_time_df.rename(columns = {0:'nb_pickups'}, inplace = True)
pickup_time_df.head()

In [ ]:
fig = px.line(x=pickup_time_df["Date/Time"], y = pickup_time_df["nb_pickups"])
fig.update_layout(
    title="Evolution of number of Uber pickups over a month",
    xaxis_title="Time",
    yaxis_title="Nb pickups"
    )
fig.show()

30 cycles are clearly seen for the month of April, one cycle per day. If we zoom in to look at what happens during a single day, we can see the morning peak at approximately 8h to 10h in the morning and the evening peak that lasts from about 17h to 22h. If we zoom in to look at the evening peak, we can see that it is in its turn consists of two peaks separated by a trough at around 20h-20h30.

For performance reasons, we will take a sample of 10000 rows from the dataframe for further analysis.

In [ ]:
df = df.sample(n=10000)
df = df.reset_index(drop=True)

Let us visualize what these pickup points look like on a map.

In [ ]:
fig = px.scatter_mapbox(
        df, 
        lat="Lat", 
        lon="Lon",
        mapbox_style="open-street-map"
)

fig.show()

As we have displayed all 10 000 points of the dataframe at once, they are very close to each other and it's difficult to see any pattern. For visualisation purposes, it might be more interesting not to visualize all pickup points known for the whole month, but to look at the daily or even hourly slots.

### Preprocessing with Pandas and feature engineering

In [ ]:
# Dropping the 'Base' column as explained before
df = df.drop('Base', axis=1)

# Sorting dataframe in chronological order
df = df.sort_values("Date/Time").reset_index(drop=True)

# Extracting features from timestamps for future use
df["Day_of_Month"] = pd.DatetimeIndex(df["Date/Time"]).day
df["Week_Number"] = df["Date/Time"].dt.isocalendar().week
df['Day_of_Week'] = pd.to_datetime(df["Date/Time"]).dt.day_name()
df['Hour'] = df["Date/Time"].dt.hour
#df['Minute'] = df["Date/Time"].dt.minute


In [ ]:
# Because of presumed daily and weekly cycles in activity, let us create a column 
# that will contain combined information on the day of the week and the hour
df["Day_of_Week_and_Hour"] = df["Day_of_Week"] + "_" + df["Hour"].astype(str)+"_hrs"

In [ ]:
"""day_of_week_df = df.groupby(by = "Day_of_Week").size().to_frame().reset_index()
day_of_week_df"""

In [ ]:
hourly_df = df.groupby(by = "Day_of_Week_and_Hour").size().to_frame().reset_index()
hourly_df.rename(columns = {0:'nb_pickups'}, inplace = True)
hourly_df.head(5)

In [ ]:
hourly_df = hourly_df.sort_values(by=['nb_pickups'], ascending=False)
hourly_df.reset_index(inplace=True, drop=True)

print("Ten hours in a week with maximum pickups: ")
display(hourly_df.head(10))

print()
print("Ten hours in a week with the fewest pickups: ")
display(hourly_df.tail(10))

We can see that late weekday nights have the fewest pickups, and weekday evening rush hours have the most pickups.

In [ ]:
#df["Year"] = pd.DatetimeIndex(df["Date/Time"]).year
#df["Month"] = pd.DatetimeIndex(df["Date/Time"]).month

In [ ]:
# Lets us look at an animated map where each frame shows pickup points in a given hour of a day
fig = px.scatter_mapbox(df, 
                        lat="Lat", lon="Lon", animation_frame='Hour', zoom=10,
                        mapbox_style="open-street-map")
fig.show()

In [ ]:
# Lets us look at an animated map where each frame shows pickup points in a given hour of a day
fig = px.scatter_mapbox(df, 
                        lat="Lat", lon="Lon", animation_frame='Day_of_Week_and_Hour', zoom=10,
                        mapbox_style="open-street-map")
fig.show()

### Preprocessing with Scikit-Learn

In [ ]:
#numeric_features = ['Lat', 'Lon', 'Day_of_Month', 'Hour', 'Minute']
numeric_features = ['Lat', 'Lon', 'Day_of_Month']
numeric_transformer = StandardScaler()

categorical_features = ['Day_of_Week', "Day_of_Week_and_Hour", 'Hour']
categorical_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
X = preprocessor.fit_transform(df)

In [ ]:
db = DBSCAN(eps=0.2, min_samples=6)

db.fit(X)
prediction_1 = db.labels_

In [ ]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print("Estimated number of clusters: %d" % n_clusters_)

In [ ]:
df["cluster"] = labels

df.tail(20)

In [ ]:
df = df[df['cluster'] != -1]
df.tail(20)

In [ ]:
fig = px.scatter_mapbox(df, lat="Lat", lon="Lon", color="cluster", zoom=10, mapbox_style="open-street-map")
fig.show()

In [ ]:
# Lets us look at an animated map where each frame shows pickup points in a given day
fig = px.scatter_mapbox(df, 
                        lat="Lat", lon="Lon", animation_frame='Day_of_Month', color="cluster",
                        mapbox_style="open-street-map")
fig.show()

In [ ]:
# Lets us look at an animated map where each frame shows pickup points in a given day
fig = px.scatter_mapbox(df, 
                        lat="Lat", lon="Lon", animation_frame='Day_of_Week', color="cluster",
                        mapbox_style="open-street-map")

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=10),
)

fig.show()

In [ ]:
# Lets us look at an animated map where each frame shows pickup points in a given hour of a day
fig = px.scatter_mapbox(df, 
                        lat="Lat", lon="Lon", animation_frame='Day_of_Week_and_Hour', zoom=10,
                        color="cluster",
                        mapbox_style="open-street-map")
fig.show()